# Liver Model Construction: Notebook-Glycogenolysis

## Setup workflow

### Import packages

In [1]:
import os
import warnings
from cobra.io.json import load_json_model as load_json_cobra_model
import escher
import mass
import numpy as np
import pandas as pd
import sympy as sym
from cobra import Model, Reaction, Metabolite
import cobra.test
from os.path import join
from mass.util import qcqa_model
from cobra import DictList
from mass import (
    MassConfiguration, MassMetabolite, MassModel,
    MassReaction, Simulation, UnitDefinition)
from mass.io.json import save_json_model as save_json_mass_model
from mass.visualization import plot_comparison, plot_time_profile
mass_config = MassConfiguration()
mass_config.irreversible_Keq = float("inf")
print("MASSpy version: {0}".format(mass.__version__))
from six import iteritems
import matplotlib.pyplot as plt
from mass.thermo import (
    ConcSolver, sample_concentrations,
    update_model_with_concentration_solution)

MASSpy version: 0.1.1


## Set paths and constants

In [2]:
seed = int(4)
#increase number of models from before
n_models = 20

mass_config = MassConfiguration()
mass_config.solver = "cplex"

### Directory paths

In [3]:
#had to remove the mass
model_dir = os.path.abspath("../mass_user/models")
maps_dir = os.path.abspath("../mass_user/maps")
data_dir = os.path.abspath("../mass_user/data")

In [4]:
# Allow Escher to close without pop-up
escher.rc['never_ask_before_quit'] = True


### Load COBRA model

In [5]:
core_model=load_json_cobra_model(filename=os.path.join(model_dir,"CoreModel.json"))
core_model

Name,None
Memory address,0x07fd822f8f410
Number of metabolites,178
Number of reactions,187
Number of groups,0
Objective expression,1.0*ATPM - 1.0*ATPM_reverse_5b752
Compartments,"m, i, c, r,"


In [6]:
# core_model.remove_reactions(EX_glc__D_c)

In [7]:
# core_model.remove_reactions([EX_glc__D_c])

In [8]:
core_model.boundary

[<Reaction EX_h_c at 0x7fd822b3db50>,
 <Reaction EX_pyr_c at 0x7fd822b3ddd0>,
 <Reaction EX_h2o_c at 0x7fd822b3d810>,
 <Reaction EX_pi_c at 0x7fd822b45890>,
 <Reaction EX_glc__D_c at 0x7fd822b45490>,
 <Reaction EX_lac__L_c at 0x7fd822b45990>,
 <Reaction EX_co2_c at 0x7fd822b45a90>,
 <Reaction EX_o2_c at 0x7fd822b45b90>,
 <Reaction EX_octa_prod_c at 0x7fd822b45c90>,
 <Reaction EX_octa_cons_c at 0x7fd822b45d90>,
 <Reaction EX_urea_c at 0x7fd822b45e90>,
 <Reaction EX_gln__L_c at 0x7fd822b45f90>,
 <Reaction EX_acetone_c at 0x7fd822b45210>,
 <Reaction EX_bhb_c at 0x7fd822e8d2d0>,
 <Reaction EX_glu__L_c at 0x7fd822e8d1d0>,
 <Reaction EX_ser__L_c at 0x7fd822e8d3d0>,
 <Reaction EX_cys__L_c at 0x7fd822e8d4d0>,
 <Reaction EX_gly_c at 0x7fd822e8d5d0>,
 <Reaction EX_glygn2_c at 0x7fd822e8d6d0>,
 <Reaction EX_Tyr_ggn_c at 0x7fd822e8d7d0>,
 <Reaction EX_ala__L_c at 0x7fd822e8d8d0>,
 <Reaction EX_nh4_c at 0x7fd822e8d9d0>,
 <Reaction EX_so3_c at 0x7fd822e8dad0>,
 <Reaction EX_etoh_c at 0x7fd822e8dbd0>

In [9]:
## adding reactions from Recon3D
R3D=load_json_cobra_model(filename=os.path.join(model_dir,"Recon3D.json"))

In [10]:
## adding reactions from glycogenloysis 
gly=load_json_cobra_model(filename=os.path.join(model_dir,"Glycogenolysis.json"))

In [11]:
## change after double checking this, maybe not needed to be added to core?
a= R3D.reactions.EX_glc__D_e
b= R3D.reactions.GLCt1
g= gly.reactions.SK_g6p_c
new_r_list=[
            a,
            b,
            g
    
]


In [12]:
for x in new_r_list:
    c= x
    core_model.add_reaction(c)

## View Escher map

In [13]:
#view all the reactions that we need build
escher_builder = escher.Builder(
    model=core_model,
    map_json=os.path.join(
        maps_dir, ".".join((
            "Glycogenolysis", "map", "json"))
    ),
    highlight_missing=True)

escher_builder


Builder(highlight_missing=True, never_ask_before_quit=True)

### Loading Measured Fluxes

In [14]:
def load_data(filepath, sheet_name):
    """Load Liver data from an excel sheet"""
    df = pd.read_excel(engine='openpyxl',
        io=filepath,
        sheet_name=sheet_name,
        index_col=0)
    return df

In [15]:
#define data sheet as latest data sheet
data_sheet=os.path.join(data_dir,"1.10-sz-data-collection.xlsx")

flux_df = load_data(
    filepath=data_sheet,
    sheet_name="Fluxes").drop("Fasting Time", axis=1).drop("Additional Notes", axis=1).drop("Unnamed: 4", axis=1).drop("Unnamed: 5", axis=1)
    ## weird anomaly, shouldn't happen * unnames:4 and 5

flux_df = flux_df.dropna() #drops rows with NaN
flux_df


,Flux(mmol/(gDW*hr))
ID,
GLCter,-0.001682
G6PPer,0.001682
G6Pter,0.001682
GLPASE1,0.000223
PGMT,0.000223
EX_glygn2_c,-0.000223
GAPD,-0.002919
PGK,0.002919
PGM,0.002919


In [16]:
v_vars = []
v_data = []

# How much flux to allow through irreversible reaction pairs
# e.g, PFK/FBP & reverse_flux_percent = 0.1, raise both fluxes by 10% of non-zero flux value
reverse_flux_percent = 0.1
irreversible_enzyme_pairs = ["PFK", "FBP"]

In [17]:
core_model.reactions

[<Reaction CSm at 0x7fd822f1bb50>,
 <Reaction ACONTm at 0x7fd822f1bb10>,
 <Reaction ICDHxm at 0x7fd822f1bb90>,
 <Reaction AKGDm at 0x7fd822f1bbd0>,
 <Reaction SUCOASm at 0x7fd822f281d0>,
 <Reaction SUCD1m at 0x7fd822f28190>,
 <Reaction FUMm at 0x7fd822f28dd0>,
 <Reaction MDHm at 0x7fd822f280d0>,
 <Reaction NADH2_u10mi at 0x7fd822f2dd90>,
 <Reaction FADH2ETC at 0x7fd822f28e10>,
 <Reaction CYOR_u10mi at 0x7fd822f36250>,
 <Reaction CYOOm2i at 0x7fd822f362d0>,
 <Reaction ATPS4mi at 0x7fd822ed0e50>,
 <Reaction Htmi at 0x7fd822f36c50>,
 <Reaction HEX1 at 0x7fd822f36210>,
 <Reaction PGI at 0x7fd822f36f90>,
 <Reaction PFK at 0x7fd822ecd810>,
 <Reaction FBA at 0x7fd822ecd610>,
 <Reaction TPI at 0x7fd822f44c90>,
 <Reaction GAPD at 0x7fd822f441d0>,
 <Reaction PGK at 0x7fd822f44210>,
 <Reaction PGM at 0x7fd822dfa110>,
 <Reaction ENO at 0x7fd822dfa590>,
 <Reaction PYK at 0x7fd822dfa2d0>,
 <Reaction ADK1 at 0x7fd822dfb090>,
 <Reaction PDHm at 0x7fd822dfbb90>,
 <Reaction LDH_L at 0x7fd822dfc810>,
 <R

In [18]:
for rid, flux in flux_df.itertuples():
    reaction = core_model.reactions.get_by_id(rid)
    print(reaction)

GLCter: glc__D_c <=> glc__D_r
G6PPer: g6p_r + h2o_r --> glc__D_r + pi_r
G6Pter: g6p_c <=> g6p_r
GLPASE1: glygn2_c + 3.0 pi_c --> dxtrn_c + 3.0 g1p_c
PGMT: g1p_c <=> g6p_c
EX_glygn2_c: glygn2_c --> 
GAPD: g3p_c + nad_c + pi_c <=> 13dpg_c + h_c + nadh_c
PGK: 3pg_c + atp_c <=> 13dpg_c + adp_c
PGM: 2pg_c <=> 3pg_c
ENO: 2pg_c <=> h2o_c + pep_c
TPI: dhap_c <=> g3p_c
FBA: fdp_c <=> dhap_c + g3p_c
FBP: fdp_c + h2o_c --> f6p_c + pi_c
PFK: atp_c + f6p_c --> adp_c + fdp_c + h_c
PGI: g6p_c <=> f6p_c
PEPCKm: gtp_m + oaa_m --> co2_m + gdp_m + pep_m
PCm: atp_m + hco3_m + pyr_m --> adp_m + h_m + oaa_m + pi_m
LDH_L: lac__L_c + nad_c <=> h_c + nadh_c + pyr_c
PYK: adp_c + h_c + pep_c --> atp_c + pyr_c


KeyError: 'ME2\xa0'

In [ ]:
core_model.reactions

In [ ]:
for rid, flux in flux_df.itertuples():
    reaction = core_model.reactions.get_by_id(rid)
    #reaction.bounds = (flux, flux)
    #set bounds for all boundary reactions 
    #uptake --> (-value, 1000)
    #secretion --> (-1000 --> value)
    #reversible --> (-1000,1000)
    if (rid == "EX_lac__L_c"): #uptake
        reaction = core_model.reactions.get_by_id(rid)
        reaction.bounds = (0,1000) 
    elif (rid == "EX_glc__L_c"): #secretion
        reaction = core_model.reactions.get_by_id(rid)
        reaction.bounds = (-1000, 0) 
    elif (rid == "EX_pyr_c"): #uptake
        reaction = core_model.reactions.get_by_id(rid)
        reaction.bounds = (0, 1000) 
    elif (rid == "EX_glygn2_c"): #uptake
        reaction = core_model.reactions.get_by_id(rid)
        reaction.bounds = (-1, 1000) 
    for irreversible_enzyme_pair in irreversible_enzyme_pairs:
        if rid in irreversible_enzyme_pair:
            flux1, flux2 = flux_df.loc[irreversible_enzyme_pairs, "Flux(mmol/(gDW*hr))"].values
            if flux1 == 0:
                flux += reverse_flux_percent * flux2 # mmol*gDW^-1*hr^-1
            if flux2 == 0:
                flux += reverse_flux_percent * flux1 # mmol*gDW^-1*hr^-1
   
    #address irreversible enzyme pair boundary reactions if necessary

    print(rid, flux)

    v_vars.append(sym.Symbol(rid))
    v_data.append(flux)
    
#print(v_vars)
#print(v_data)


In [ ]:
core_model.reactions

In [ ]:
# Make symbolic for optlang objective 
v_vars = sym.Matrix(v_vars)
v_data = sym.Matrix(v_data)

F = sym.Matrix(2 * sym.eye(len(v_vars)))

objective = 0.5 * v_vars.T * F * v_vars  - (2 * v_data).T * v_vars
core_model.objective = objective[0]
core_model.objective_direction = "min"

flux_solution = core_model.optimize()

In [ ]:
flux_solution

In [ ]:
flux_comparison_fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(5, 5))
plot_comparison(
    x=flux_df["Flux (mmol/gDW*hr)"], y=flux_solution, compare="fluxes",
    observable=[rid for rid in flux_df.index], ax=ax,
    legend="right outside", plot_function="plot",
    xlim=(-0.005,0.005), ylim=(-0.005, 0.005),
    xy_line=True,
    xy_legend="best", xlabel="Measured [mmol/(gDW * h)]", ylabel="Adjusted [mmol/(gDW * h)]")

flux_comparison_fig.tight_layout()
#check directionality 


## Create MASS model

In [ ]:
glycogenolysis = MassModel("Glycogenolysis", array_type='DataFrame', dtype=np.int64)
glycogenolysis

### Define reactions

In [ ]:
#add reactions one at a time
reaction_list=[]
reaction_list = ['HEX1',
                 'GLCt1'
#                  'G6Pter',
#                  'G6PPer'
#                 #'PGI',
#                 #'FBP',
#                 'PFK',
#                 'FBA',
#                 'TPI',
#                 'GAPD',
#                 'PGK',
#                 'PGM',
#                 'ENO',
#                 #'PEPtm',
#                 #'PEPCKm',
#                 #'LDH_L',
#                 'PYK'
#                 #'PCm',
#                 #'PYRt2m'
                ]

# a= R3D.reactions.EX_glc__D_e
# b= R3D.reactions.GLCt1
# g= gly.reactions.SK_g6p_c
# new_r_list=[
#             a,
#             b,
#             g
    
# ]


In [ ]:
# Convert cobra.Reactions to mass.MassReactions
for rid in reaction_list:
    reaction = core_model.reactions.get_by_id(rid)
    glycogenolysis.add_reactions([MassReaction(reaction)])

In [ ]:
# for x in new_r_list:
#     c= x
#     core_model.add_reaction(c)

##  Set Fluxes

In [ ]:
for reaction, flux in flux_solution[reaction_list].iteritems():
    reaction = glycogenolysis.reactions.get_by_id(reaction)
    reaction.steady_state_flux = flux * 1278.605 / 1000 
    # mmol/gdW*hr --> mmol/L * hr ---> mol/L*hr
    print("{0}: {1}".format(reaction.flux_symbol_str,
                            reaction.steady_state_flux))

## Set Keqs

In [ ]:
# Load equilibrium constants
Keq_df = pd.read_excel(engine='openpyxl',
    io=data_sheet,
    sheet_name="Keqs",
    index_col=0).drop("Reference", axis=1).drop("Stoichiometry", axis=1).drop('Keq unit', axis=1)

Keq_df = Keq_df.dropna() #drops rows with NaN
Keq_df

Keq_df
#Check the M units of Keq 



In [ ]:
Keq_list =[]
for rid, Keq in Keq_df.itertuples():
    if rid in reaction_list: 
        reaction = glycogenolysis.reactions.get_by_id(rid)
        reaction.Keq = Keq
        Keq_list.append(reaction.Keq)
        print("{0}: {1}".format(reaction.Keq_str, Keq))

## Set H2O/H Constants

In [ ]:
# Set concentrations of hydrogen, water as fixed
# Assume hydrogen in all compartments are fixed 
for metabolite in [
#     "h2o_c",
    "h_c"
#     "glc__D_e",
#     'h20_r'
    
       #'h_i'
       #'h_m' 
    #remove h_i and h_m if not mitochondrial reactions
                  ]:
    metabolite = glycogenolysis.metabolites.get_by_id(metabolite)
    metabolite.fixed = True

## Obtain Concentrations

In [ ]:
#Load Concentration Data
conc_df = load_data(
    filepath=data_sheet,
    sheet_name="Concentrations").drop("Concentration (mM)", axis=1)
#conc_df = conc_df.drop("Name", axis=1)
conc_df = conc_df.drop("Additional Notes", axis=1)
conc_df = conc_df.dropna() #drops rows with NaN
conc_df

In [ ]:
#Set Concentrations

metabolite_list = [
# HEX1    
    'glc__D_c',
    'g6p_c',
    'adp_c',
    'atp_c',
    #'pi_c',
    'h_c',
# GLCt1
    'glc__D_e'
# #G6Pter
#             'g6p_r',
# #G6PPer 
#             'h2o_r',
#             'glc__D_r',
#             'pi_r'
            
                  ]

In [ ]:
met_list = []
for mid, name, conc in conc_df.itertuples():
    if mid in metabolite_list: 
        metabolite = glycogenolysis.metabolites.get_by_id(mid)
        metabolite.name = name
        
        metabolite.ic = conc #data sheet is in M units
        met_list.append(metabolite.ic)

In [ ]:
for metabolite in [
    #'h_i',
    #'h_m',
    "h_c", 
    #"h2o_c"
#'h20_r'
]:
    metabolite = glycogenolysis.metabolites.get_by_id(metabolite)
    metabolite.ic = 1 #M
    
glycogenolysis.metabolites.glc__D_e.ic= glycogenolysis.metabolites.glc__D_c.ic

for metabolite in glycogenolysis.metabolites:
    print("{0}: {1}".format(metabolite.id, metabolite.ic))

In [ ]:
#Formulate QP minimization list for concentrations
## double check this?
conc_solver = ConcSolver(
    glycogenolysis, # bc only want metabolites in glycogenolysis 
    excluded_metabolites=[
        #'h_i', 'h_m', 'co2_m', 'gdp_m', 'gtp_m', 'hco3_m'
        "h_c", 
        #"h2o_c",'13dpg_c'
    ],
    constraint_buffer=1)

conc_solver.setup_feasible_qp_problem(
fixed_conc_bounds=list(glycogenolysis.fixed))
fixed_Keq_bounds=glycogenolysis.reactions.list_attr("Keq_str")
    
#assumption: provide uncertainity to allow for a variance for the Keqs rather than staying fixed
#conc_solver.setup_feasible_qp_problem(Keq_percent_deviation=0.1)

conc_solution = conc_solver.optimize()
conc_solution

In [ ]:
#Compare Results
conc_comparison_fig, ax = plt.subplots(nrows=1, ncols=1,
                                       figsize=(5, 5))

plot_comparison(
    x=conc_df["Concentration (mM)"], y=conc_solution,
    compare="concentrations",
    observable=[mid for mid in conc_df.index], ax=ax,
    legend="right outside", plot_function="loglog",
    xlim=(0, 20), ylim=(0, 20), xy_line=True,
    xy_legend="best", xlabel="Initial [mol/L]", ylabel="Adjusted [mol/L]")

conc_comparison_fig.tight_layout()
update_model_with_concentration_solution(
    glycogenolysis, conc_solution, concentrations=True, inplace=True);

## Sample Concentrations

In [ ]:
conc_solver.setup_sampling_problem(
    fixed_conc_bounds=list(glycogenolysis.fixed),
    fixed_Keq_bounds=glycogenolysis.reactions.list_attr("Keq_str"))
for variable in conc_solver.variables:
    try:
        met = glycogenolysis.metabolites.get_by_id(variable.name)
        variable.lb, variable.ub = np.log([met.ic / 10, met.ic * 10])
    except:
        pass
conc_samples = sample_concentrations(conc_solver, n=n_models, seed=seed)
conc_samples.head()

## Set concentrations and balance models with pseudoreactions

In [ ]:
models_for_ensemble = []
for idx, conc_sample in conc_samples.iterrows():
    # Make copy of new model
    new_model = glycogenolysis.copy()
    new_model.id += "_C{0:d}".format(idx)
    # Get concentration sample and update model with sample
    new_model.update_initial_conditions(conc_sample.to_dict())

    # Determine imbalances in the reduced network
    fluxes = np.array(list(new_model.steady_state_fluxes.values()))
    imbalanced_metabolites = new_model.S.dot(fluxes)

    # Iterate through metabolites
    for mid, imbalance in imbalanced_metabolites.iteritems():
        # Ignore balanced metabolites
        if imbalance == 0:
            continue
        # Get metabolite object
        met = new_model.metabolites.get_by_id(mid)

        # Add boundary reactions for imbalanced metabolites
        boundary_type = "sink"    
        # Add boundary reaction with imbalance as flux value
        boundary_reaction = new_model.add_boundary(
            mid, boundary_type, boundary_condition=met.ic)
        
        boundary_reaction.Keq = 1
        if imbalance < 0:
            boundary_reaction.reverse_stoichiometry(inplace=True)
            imbalance = -imbalance

        boundary_reaction.kf = imbalance / met.ic
        boundary_reaction.steady_state_flux = imbalance
        try:
            # Update PERCs
            new_model.calculate_PERCs(
                fluxes={
                    r: v for r, v in new_model.steady_state_fluxes.items()
                    if not r.boundary},
                update_reactions=True)
        except:
            print("Negative PERCs for {0}".format(new_model.id))
            continue
    models_for_ensemble.append(new_model)
print("Number of models in ensemble: {0:d}".format(
    len(models_for_ensemble)))
print(models_for_ensemble)

#### Ensure model is in steady state and export

In [ ]:
## determine imbalances in the reduced network
fluxes = np.array(list(glycogenolysis.steady_state_fluxes.values()))
imbalanced_metabolites = glycogenolysis.S.dot(fluxes)

# Iterate through metabolites
for mid, imbalance in imbalanced_metabolites.iteritems():
    # Ignore balanced metabolites
    if imbalance == 0:
        continue
    # Get metabolite object
    met = glycogenolysis.metabolites.get_by_id(mid)

    # Add boundary reactions for imbalanced metabolites
    boundary_type = "sink"    
    # Add boundary reaction with imbalance as flux value
    boundary_reaction = glycogenolysis.add_boundary(
        mid, boundary_type, boundary_condition=met.ic)

    boundary_reaction.Keq = 1
    if imbalance < 0:
        boundary_reaction.reverse_stoichiometry(inplace=True)
        imbalance = -imbalance

    boundary_reaction.kf = imbalance / met.ic
    boundary_reaction.steady_state_flux = imbalance
    try:
        # Update PERCs
        glycogenolysis.calculate_PERCs(
            fluxes={
                r: v for r, v in glycogenolysis.steady_state_fluxes.items()
                if not r.boundary},
            update_reactions=True)
    except:
        print("Negative PERCs for {0}".format(new_model.id))
        continue

In [ ]:
## balancing charges on reactions manually

glycogenolysis.metabolites.h_c.charge=1

In [ ]:
qcqa_model(glycogenolysis, parameters=True, concentrations=True,
          fluxes=True, superfluous=True, elemental=True)


In [ ]:
# Whether to print messages as loop occurs. 
verbose = False
# Ensure all models are simulated to a long time and that
# there are enough time points to determine if model reached steady state.
tfinal = 1e4

sim = Simulation(reference_model=models_for_ensemble[0], verbose=verbose)
sim.add_models(models_for_ensemble[1:], verbose=verbose, disable_safe_load=True)
sim.integrator.absolute_tolerance = 1e-15
sim.integrator.relative_tolerance = 1e-9

models_to_export = []
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    for model in models_for_ensemble:
        # Attempt to determine steady state
        conc_sol, flux_sol = sim.find_steady_state(
            models=model, strategy="simulate", update_values=True,
            tfinal=tfinal, 
            verbose=verbose)
        if conc_sol and flux_sol:
            continue
        print(str(model) + " first attempt failed.")
        # First attempt was not successful, lengthen time for second attempt
        # Attempt to determine steady state
        conc_sol, flux_sol = sim.find_steady_state(
            models=model, strategy="simulate", update_values=True,
            tfinal=tfinal * 10, 
            verbose=verbose)
        if conc_sol and flux_sol:
            continue
        print(str(model) + " second attempt failed.")
        # Second attempt was not successful, lengthen time for second attempt
        # Attempt to determine steady state
        conc_sol, flux_sol = sim.find_steady_state(
            models=model, strategy="simulate", update_values=True,
            tfinal=tfinal * 100, 
            verbose=verbose)
        if conc_sol and flux_sol:
            continue
        print(str(model) + " third attempt failed. Eliminating model.")
        # Third attempt was not successful, eliminate the model
        sim.remove_models(models=model, verbose=verbose)

In [ ]:
glycogenolysis.genes.remove('3098_AT4')
glycogenolysis.genes.remove('2542_AT1')

In [ ]:
###RUN THIS FOR TIME PROFILE
# Setup simulation object
sim = Simulation(glycogenolysis, verbose=False)
# Simulate from 0 to 1000 with 10001 points in the output
conc_sol, flux_sol = sim.simulate(glycogenolysis, time=(0, 1e3, 1e4 + 1))
# Quickly render and display time profiles
conc_sol.view_time_profile()

In [ ]:
fig_1, axes = plt.subplots(nrows=2, ncols=1, figsize=(10, 6));
(ax1, ax2) = axes.flatten()
L_FONT = {"size": "large"}
plot_time_profile(
    conc_sol, ax=ax1, legend="right outside",
    plot_function="semilogx",
    xlabel="Time [hr]", ylabel="Concentration [mM]",
    title=("Concentration Profile", L_FONT));

plot_time_profile(
    flux_sol, ax=ax2, legend="right outside",
    plot_function="semilogx",
    xlabel="Time [hr]", ylabel="Flux [mol/L*hr]",
    title=("Flux Profile", L_FONT));
fig_1.tight_layout()

